GT4Py - GridTools for Python

Copyright (c) 2014-2023, ETH Zurich
All rights reserved.

This file is part the GT4Py project and the GridTools framework.
GT4Py is free software: you can redistribute it and/or modify it under
the terms of the GNU General Public License as published by the
Free Software Foundation, either version 3 of the License, or any later
version. See the LICENSE.txt file at the top-level directory of this
distribution for a copy of the license or check <https://www.gnu.org/licenses/>.

SPDX-License-Identifier: GPL-3.0-or-later

# Comparison gt4py.cartesian vs. gt4py.next

Imports

In [4]:
import numpy as np

nx = 32
ny = 32
nz = 1
dtype = np.float64

gt4py.cartesian
--

In [5]:
import gt4py.storage as storage
import gt4py.cartesian.gtscript as gtscript

backend = "numpy"
# backend = "gt:cpu_ifirst"

@gtscript.stencil(backend=backend)
def lap_cartesian(
    inp: gtscript.Field[dtype],
    out: gtscript.Field[dtype],
):
    with computation(PARALLEL), interval(...):
        out = -4.0 * inp[0, 0, 0] + inp[-1, 0, 0] + inp[1, 0, 0] + inp[0, -1, 0] + inp[0, 1, 0]

inp = storage.from_array(np.fromfunction(lambda x, y, z: x**2+y**2, shape=(nx, ny, nz)), dtype, backend=backend)
out_cartesian = storage.zeros((nx, ny, nz), dtype, backend=backend)

lap_cartesian(inp=inp, out=out_cartesian, origin=(1, 1, 0), domain=(nx-2, ny-2, nz))

In [6]:
from gt4py.next.common import Dimension, Field
from gt4py.next.ffront.decorator import field_operator, program
from gt4py.next.ffront.fbuiltins import FieldOffset
from gt4py.next.iterator.embedded import np_as_located_field as with_dimensions
from gt4py.next.program_processors.runners.roundtrip import executor as itir_embedded
from gt4py.next.program_processors.runners.gtfn_cpu import run_gtfn as cpu_backend

backend = itir_embedded
# backend = cpu_backend

IDim = Dimension("IDim")
JDim = Dimension("JDim")
KDim = Dimension("KDim")

I = FieldOffset("I", source=IDim, target=(IDim,))
J = FieldOffset("J", source=JDim, target=(JDim,))

@field_operator
def lap_next(inp: Field[[IDim, JDim, KDim], dtype]) -> Field[[IDim, JDim, KDim], dtype]:
    return -4.0 * inp + inp(I[-1]) + inp(I[1]) + inp(J[-1]) + inp(J[1])

@program(backend=backend)
def lap_next_program(inp: Field[[IDim, JDim, KDim], dtype], out: Field[[IDim, JDim, KDim], dtype]):
    lap_next(inp, out=out[1:-1, 1:-1, :])

inp_next = with_dimensions(IDim,JDim,KDim)(inp)
out_next = with_dimensions(IDim, JDim, KDim)(np.zeros(shape=(nx, ny, nz)))

lap_next_program(inp_next, out_next, offset_provider={"I": IDim, "J": JDim})

In [7]:
assert np.allclose(out_cartesian, out_next)